In [1]:
import sqlite3

import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
__file__ = %pwd
DATA = Path(__file__).parent / "data"
LTEL = DATA / "LTEL"
DATABASE = DATA / "databases" / "exploration.db"

In [3]:
ltel24 = pd.read_csv(LTEL / "LtelDownload2324.txt", sep="\t")
ltel23 = pd.read_csv(LTEL / "LtelDownload2223.txt", sep="\t")
ltel22 = pd.read_csv(LTEL / "LtelDownload2122.txt", sep="\t")
ltel21 = pd.read_csv(LTEL / "LtelDownload2021.txt", sep="\t")
ltel20 = pd.read_csv(LTEL / "LtelDownload1920.txt", sep="\t")
ltel19 = pd.read_csv(LTEL / "LtelDownload1819.txt", sep="\t")
ltel18 = pd.read_csv(LTEL / "LtelDownload1718.txt", sep="\t")
ltel17 = pd.read_csv(LTEL / "LtelDownload1617.txt", sep="\t")

# Only include actual schools.
ltel24 = ltel24[(ltel24["AggLevel"] == "S") & (ltel24["SchoolCode"] != 0) & (ltel24["SchoolCode"] != 1) & (ltel24["Charter"] == "N")]
ltel23 = ltel23[(ltel23["AggLevel"] == "S") & (ltel23["SchoolCode"] != 0) & (ltel23["SchoolCode"] != 1) & (ltel23["Charter"] == "N")]
ltel22 = ltel22[(ltel22["AggLevel"] == "S") & (ltel22["SchoolCode"] != 0) & (ltel22["SchoolCode"] != 1) & (ltel22["Charter"] == "N")]
ltel21 = ltel21[(ltel21["AggLevel"] == "S") & (ltel21["SchoolCode"] != 0) & (ltel21["SchoolCode"] != 1) & (ltel21["Charter"] == "N")]
ltel20 = ltel20[(ltel20["AggLevel"] == "S") & (ltel20["SchoolCode"] != 0) & (ltel20["SchoolCode"] != 1) & (ltel20["Charter"] == "N")]
ltel19 = ltel19[(ltel19["AggLevel"] == "S") & (ltel19["SchoolCode"] != 0) & (ltel19["SchoolCode"] != 1) & (ltel19["Charter"] == "N")]
ltel18 = ltel18[(ltel18["AggLevel"] == "S") & (ltel18["SchoolCode"] != 0) & (ltel18["SchoolCode"] != 1) & (ltel18["Charter"] == "N")]
ltel17 = ltel17[(ltel17["AggLevel"] == "S") & (ltel17["SchoolCode"] != 0) & (ltel17["SchoolCode"] != 1) & (ltel17["Charter"] == "N")]

# Only include High Schoolers in the count.
ltel24 = ltel24[(ltel24["Grade"] == "09") | (ltel24["Grade"] == "10") | (ltel24["Grade"] == "11") | (ltel24["Grade"] == "12")]
ltel23 = ltel23[(ltel23["Grade"] == "09") | (ltel23["Grade"] == "10") | (ltel23["Grade"] == "11") | (ltel23["Grade"] == "12")]
ltel22 = ltel22[(ltel22["Grade"] == "09") | (ltel22["Grade"] == "10") | (ltel22["Grade"] == "11") | (ltel22["Grade"] == "12")]
ltel21 = ltel21[(ltel21["Grade"] == "09") | (ltel21["Grade"] == "10") | (ltel21["Grade"] == "11") | (ltel21["Grade"] == "12")]
ltel20 = ltel20[(ltel20["Grade"] == "09") | (ltel20["Grade"] == "10") | (ltel20["Grade"] == "11") | (ltel20["Grade"] == "12")]
ltel19 = ltel19[(ltel19["Grade"] == "09") | (ltel19["Grade"] == "10") | (ltel19["Grade"] == "11") | (ltel19["Grade"] == "12")]
ltel18 = ltel18[(ltel18["Grade"] == "09") | (ltel18["Grade"] == "10") | (ltel18["Grade"] == "11") | (ltel18["Grade"] == "12")]
ltel17 = ltel17[(ltel17["Grade"] == "09") | (ltel17["Grade"] == "10") | (ltel17["Grade"] == "11") | (ltel17["Grade"] == "12")]

# Gender is already taken into account via ACGR etc.
ltel24 = ltel24[(ltel24["Gender"] == "ALL")]
ltel23 = ltel23[(ltel23["Gender"] == "ALL")]
ltel22 = ltel22[(ltel22["Gender"] == "ALL")]
ltel21 = ltel21[(ltel21["Gender"] == "ALL")]
ltel20 = ltel20[(ltel20["Gender"] == "ALL")]
ltel19 = ltel19[(ltel19["Gender"] == "ALL")]
ltel18 = ltel18[(ltel18["Gender"] == "ALL")]
ltel17 = ltel17[(ltel17["Gender"] == "ALL")]

C:\Users\Penta\AppData\Local\Temp\ipykernel_39000\2977555910.py:1: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  ltel24 = pd.read_csv(LTEL / "LtelDownload2324.txt", sep="\t")
C:\Users\Penta\AppData\Local\Temp\ipykernel_39000\2977555910.py:2: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  ltel23 = pd.read_csv(LTEL / "LtelDownload2223.txt", sep="\t")
C:\Users\Penta\AppData\Local\Temp\ipykernel_39000\2977555910.py:3: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  ltel22 = pd.read_csv(LTEL / "LtelDownload2122.txt", sep="\t")
C:\Users\Penta\AppData\Local\Temp\ipykernel_39000\2977555910.py:4: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  ltel21 = pd.read_csv(LTEL / "LtelDownload2021.txt", sep="\t")
C:\Users\Penta\AppData\Local\Temp\ipykernel_39000\2977555910.py:5: D

In [4]:
def merge_codes(county_code: float, district_code: float, school_code: float) -> int:
    cds_code = 0
    if not np.isnan(county_code):
        cds_code += county_code * 1e12
    if not np.isnan(district_code):
        cds_code += district_code * 1e7
    if not np.isnan(school_code):
        cds_code += school_code
    return int(cds_code)

In [5]:
ltel24["cdsCode"] = ltel24.apply(lambda row: merge_codes(row["CountyCode"], row["DistrictCode"], row["SchoolCode"]), axis=1)
ltel23["cdsCode"] = ltel23.apply(lambda row: merge_codes(row["CountyCode"], row["DistrictCode"], row["SchoolCode"]), axis=1)
ltel22["cdsCode"] = ltel22.apply(lambda row: merge_codes(row["CountyCode"], row["DistrictCode"], row["SchoolCode"]), axis=1)
ltel21["cdsCode"] = ltel21.apply(lambda row: merge_codes(row["CountyCode"], row["DistrictCode"], row["SchoolCode"]), axis=1)
ltel20["cdsCode"] = ltel20.apply(lambda row: merge_codes(row["CountyCode"], row["DistrictCode"], row["SchoolCode"]), axis=1)
ltel19["cdsCode"] = ltel19.apply(lambda row: merge_codes(row["CountyCode"], row["DistrictCode"], row["SchoolCode"]), axis=1)
ltel18["cdsCode"] = ltel18.apply(lambda row: merge_codes(row["CountyCode"], row["DistrictCode"], row["SchoolCode"]), axis=1)
ltel17["cdsCode"] = ltel17.apply(lambda row: merge_codes(row["CountyCode"], row["DistrictCode"], row["SchoolCode"]), axis=1)

ltel24[["cdsCode", "CountyCode", "DistrictCode", "SchoolCode"]]

,cdsCode,CountyCode,DistrictCode,SchoolCode
170446,1100170130401,1.0,10017.0,130401.0
170448,1100170130419,1.0,10017.0,130419.0
170449,1100170130419,1.0,10017.0,130419.0
170465,1316170131763,1.0,31617.0,131763.0
170475,1100170130401,1.0,10017.0,130401.0
...,...,...,...,...
236529,58727365830054,58.0,72736.0,5830054.0
236530,58727365830096,58.0,72736.0,5830096.0
236533,58727365835202,58.0,72736.0,5835202.0
236534,58727365835202,58.0,72736.0,5835202.0


In [6]:
ltel24.columns

Index(['AcademicYear', 'CountyCode', 'DistrictCode', 'SchoolCode',
       'CountyName', 'DistrictName', 'SchoolName', 'Charter', 'AggLevel',
       'Grade', 'Gender', 'EO', 'IFEP', 'EL', 'RFEP', 'TBD', 'TotalEnrollment',
       'AR', 'LTEL', 'EL4+', 'EL03Y', 'EL45Y', 'EL6+Y', 'Total-EE', 'cdsCode'],
      dtype='object')

In [7]:
ltel = pd.concat([ltel24, ltel23, ltel22, ltel21,
                    ltel20, ltel19, ltel18, ltel17]).drop_duplicates()
ltel = ltel[["AcademicYear", "cdsCode", "CountyCode", "DistrictCode", "SchoolCode",
             "Grade", "EO", "IFEP", "EL", "RFEP", "TBD", "TotalEnrollment",
             "AR", "LTEL", "EL4+", "EL03Y", "EL45Y", "EL6+Y", "Total-EE"]]
ltel

,AcademicYear,cdsCode,CountyCode,DistrictCode,SchoolCode,Grade,EO,IFEP,EL,RFEP,TBD,TotalEnrollment,AR,LTEL,EL4+,EL03Y,EL45Y,EL6+Y,Total-EE
170446,2023-24,1100170130401,1.0,10017.0,130401.0,11,5,0,3,0,0,8,2,1,0,0,2,1,3
170448,2023-24,1100170130419,1.0,10017.0,130419.0,10,3,0,2,4,0,9,0,1,1,0,0,2,6
170449,2023-24,1100170130419,1.0,10017.0,130419.0,12,1,0,4,4,0,9,0,3,0,1,0,3,8
170465,2023-24,1316170131763,1.0,31617.0,131763.0,11,37,0,1,0,0,40,0,1,0,0,0,1,1
170475,2023-24,1100170130401,1.0,10017.0,130401.0,12,10,0,0,1,0,11,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210059,2016-17,58727365835202,58.0,72736.0,5835202.0,11,166,0,18,31,0,215,1,9,8,0,1,17,49
210088,2016-17,58727690123570,58.0,72769.0,123570.0,11,1,0,0,1,0,2,0,0,0,0,0,0,1
210089,2016-17,58727695838305,58.0,72769.0,5838305.0,10,163,0,8,19,0,190,1,5,0,2,1,5,27
210090,2016-17,58727695838305,58.0,72769.0,5838305.0,12,136,2,4,17,0,159,0,2,1,1,0,3,21


In [8]:
con = sqlite3.connect(DATABASE)
ltel.to_sql("LTEL", con, index=False)
con.close()